In [1]:
import pandas as pd
import os

In [2]:
os.chdir(r"C:\Users\Aggy\AIJobtl\H-1B_DATA")

In [3]:
def process_h1b_data(year):
    """
    Reads and processes H1B DataHub export for a given year.
    
    Parameters:
        year (int): The year of the H1B dataset
        
    Returns:
        DataFrame: A cleaned and aggregated DataFrame for that year
    """
    # Read file
    filename = f"h1b_datahubexport-{year}.csv"
    df = pd.read_csv(filename)
    
    # Drop rows with missing values
    df = df.dropna()
    
    # Group and aggregate
    grouped = df.groupby('Employer').agg(
        initial_approvals=('Initial Approval', 'sum'),
        states=('State', 'first'),
        cities=('City', 'first'),
        zip_codes=('ZIP', 'first')
    ).reset_index()
    
    # Remove employers with zero approvals
    grouped = grouped[grouped['initial_approvals'] != 0]
    
    print(f"{year} data shape after processing: {grouped.shape}")
    return grouped


In [6]:
# Process and combine data for 2020–2025
all_years = pd.concat([process_h1b_data(y) for y in range(2020, 2024)], ignore_index=True)

# Now group again by Employer to merge duplicates across years
H1B_final = all_years.groupby('Employer').agg(
    total_approvals=('initial_approvals', 'sum'),
    states=('states', 'first'),
    cities=('cities', 'first'),
    zip_codes=('zip_codes', 'first')
).reset_index()

print("Final merged dataset shape:", H1B_final.shape)
H1B_final.head()

2020 data shape after processing: (25414, 5)
2021 data shape after processing: (25340, 5)
2022 data shape after processing: (28543, 5)
2023 data shape after processing: (10104, 5)
Final merged dataset shape: (60993, 5)


,Employer,total_approvals,states,cities,zip_codes
0,0 NORTH AVE WAKEFIELD LLC,1,MA,WAKEFIELD,1880.0
1,01INTERACTIVE INC,3,CA,CITY OF INDUSTRY,91745.0
2,02908 HOLDCO LLC,1,RI,PROVIDENCE,2908.0
3,05MEDIA INC,1,CA,SAN DIEGO,92130.0
4,0965688 BC LTD DBA PROCOGIA,5,WA,BELLEVUE,98006.0


In [7]:
# Save to CSV
output_filename = "H1B_2020_2023_summary.csv"
H1B_final.to_csv(output_filename, index=False)